In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import DBSCAN
from sklearn.metrics import silhouette_score
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import re

In [2]:
da = pd.read_excel('다나와 상품.xlsx')
da = da.drop_duplicates(subset=['name.1'])

co = pd.read_excel('커뮤니티 상품 DB.xlsx')
co = co.drop_duplicates(subset=['title'])

def preprocess_text(text):
    if isinstance(text, str):
        return re.sub(r'[^\s\d\w]', '', text)
    return ''

da['name.1'] = da['name.1'].apply(preprocess_text)
co['title'] = co['title'].apply(preprocess_text)

da['name.1'] = da['name.1'].fillna('').astype(str)
co['title'] = co['title'].fillna('').astype(str)

In [3]:
texts = da['name.1'].tolist() + co['title'].tolist()

In [4]:
# TF-IDF 벡터화
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(texts)


In [5]:
from sklearn.cluster import DBSCAN

# 최적의 eps와 min_samples 설정
optimal_eps = 0.5  # K-거리 그래프에서 결정된 값
# min_samples = 5  # 일반적으로 데이터의 차원 수 + 1 이상
min_samples = 3

# DBSCAN 클러스터링
dbscan = DBSCAN(eps=optimal_eps, min_samples=min_samples)
clusters = dbscan.fit_predict(X)

# 클러스터링 결과 평가
num_clusters = len(set(clusters)) - (1 if -1 in clusters else 0)
print(f'클러스터 수: {num_clusters}')




클러스터 수: 1182


In [6]:
# 실루엣 스코어
if num_clusters > 1:
     score = silhouette_score(X, clusters)
     print(f'실루엣 스코어: {score}')
else:
     print('측정되지 않음')

실루엣 스코어: -0.27842722198667225


In [7]:
clustered_texts = pd.DataFrame({'text': texts, 'cluster': clusters})
for cluster_num in range(num_clusters):
    print(f"\nCluster {cluster_num}:")
    cluster_texts = clustered_texts[clustered_texts['cluster'] == cluster_num]['text'].values
    for text in cluster_texts:  
        print(text)
    
    



Cluster 0:
2021 울트라기어 17UD70PPX76K 16GB램
2021 울트라기어 17UD70PPX76K 64GB램
2021 울트라기어 17UD70PPX76K WIN10
2021 울트라기어 17UD70PPX76K WIN10 16GB램
2021 울트라기어 17UD70PPX76K WIN10 64GB램

Cluster 1:
2022 그램1412세대 14ZD90QGX50K
2022 그램1412세대 14ZD90QGX50K WIN10
2022 그램1412세대 14ZD90QGX50K WIN11

Cluster 2:
2022 그램1412세대 14ZD90QGX56K
2022 그램1412세대 14ZD90QGX56K WIN10
2022 그램1412세대 14ZD90QGX56K WIN11

Cluster 3:
2022 그램1512세대 15ZD90QGX56K
2022 그램1512세대 15ZD90QGX56K WIN10
2022 그램1512세대 15ZD90QGX56K WIN11

Cluster 4:
2022 그램1512세대 15ZD90QGX76K
2022 그램1512세대 15ZD90QGX76K WIN10
2022 그램1512세대 15ZD90QGX76K WIN11

Cluster 5:
2022 그램1512세대 15ZD95QGX56K
2022 그램1512세대 15ZD95QGX56K WIN10
2022 그램1512세대 15ZD95QGX56K WIN11

Cluster 6:
2022 그램1612세대 16ZD90QGX56K
2022 그램1612세대 16ZD90QGX56K WIN10
2022 그램1612세대 16ZD90QGX56K WIN11

Cluster 7:
2022 그램1612세대 16ZD95QGX56K
2022 그램1612세대 16ZD95QGX56K WIN10
2022 그램1612세대 16ZD95QGX56K WIN11

Cluster 8:
2022 그램1712세대 17ZD90QEX56K
2022 그램1712세대 17ZD90QEX56K WIN10
2022 그램1712세대 17ZD9

In [8]:
with open('클러스터 결과 min_samples = 3.txt', 'w', encoding='utf-8') as file:
    for cluster_num in range(num_clusters):
        file.write(f"\nCluster {cluster_num}:\n")
        cluster_texts = clustered_texts[clustered_texts['cluster'] == cluster_num]['text'].values
        for text in cluster_texts:
            file.write(text + '\n')